<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/Impacto_nulos_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica limpieza de datos


## 0. importar librerias

In [1]:
#Importamos las librerias que usaremos en este practica

import pandas as pd
import numpy as np
import seaborn as sns

sns.set_theme(style="whitegrid")

## 1. Cargar el data set

# Usando google colab podemos cargar el data set de dos formas, podemos usar la libreria de drive de google o podemos subir el archivo al colab.

### 1.1 Usando la libreria de drive

### 1.2 Cargando el archivo a google colab


*Otra* forma de usar el data set es subiendolo al google colab, para esto hacemos click en el icono de la carpeta que se encuentra a la izquierda.

Una vez abierta la pestaña de archivos, hacemos click en el primer icono de la parte superior de la nueva pestaña, lo cual nos permitira subir nuestro dataset a la maquina virtual.

Ahora, usamos el siguiente comando para guardarlo en una variable

In [3]:
df = pd.read_csv('./data/diabetes-NAN.csv',na_values='?')

## 2. Observaciones basicas del dataset

### Tamaño del dataset

In [4]:
df.shape

(768, 9)

### información general

Podemos observar el nombre de las columnas, si tienen nulos y cual es su tipo de dato

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               766 non-null    float64
 1   Glucose                   759 non-null    float64
 2   BloodPressure             761 non-null    float64
 3   SkinThickness             759 non-null    float64
 4   Insulin                   762 non-null    float64
 5   BMI                       761 non-null    float64
 6   DiabetesPedigreeFunction  766 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(7), int64(2)
memory usage: 54.1 KB


Gracias a esta información, podemos ver que tenemos una pequeña cantidad de datos nulos

### Vista preliminar

Observamos los primeros y ultimos 5 datos del data set

In [6]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50,1
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31,0
2,8.0,NaN,64.0,0.0,0.0,23.3,0.672,32,1
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0.0,NaN,NaN,35.0,168.0,43.1,2.288,33,1


In [7]:
df.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27,0
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47,1
767,1.0,93.0,70.0,31.0,0.0,30.4,0.315,23,0


# 3.1 Clasificación con datos nulos

In [8]:
#Escojamos nuestras variables predictoras
predictores = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
#Escojamos que es lo que queremos encontrar
objetivo= 'Outcome'

#Recordemos que en nuestra variable Y va lo que queremos encontrar/predecir.
#Mientras en la vairable X van las variables que usaremos para encontrar a Y
y= df[objetivo].values
X= df[predictores].values

In [9]:
from sklearn.model_selection import train_test_split
# Crear conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(614, 8)
(154, 8)
(614,)
(154,)


Si intentamos correr este codigo, veremos que la regresión logisitica no puede lidear con valores nulos, así que usaremos otro modelo.

In [10]:
# Se importa el modelo
from sklearn.linear_model import LogisticRegression

#Se crea el modelo
log_reg = LogisticRegression(solver='lbfgs',max_iter=100)
#se entrena el modelo
log_reg.fit(X_train, y_train)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [11]:
from sklearn.ensemble import (HistGradientBoostingClassifier)

hgbm = HistGradientBoostingClassifier(random_state=42)
hgbm.fit(X_train, y_train)
hgbm.score(X_test, y_test)

0.7272727272727273

Nuestro resultado fue de 0.72, que no esta nada mal, ahora veamos que pasa si nos encargamos de los valores nulos

# 3.2 Clasificación sin datos nulos

In [12]:
#veamos cuantos nan tenemos en columnas
df.isna().sum()

Pregnancies                 2
Glucose                     9
BloodPressure               7
SkinThickness               9
Insulin                     6
BMI                         7
DiabetesPedigreeFunction    2
Age                         0
Outcome                     0
dtype: int64

In [13]:
#veamos cuantos nan tenemos en total
df.isna().sum().sum()

42

Podemos ver, que solo 42 de las 768 observaciones tienen valores nulos, así que en este caso borraremos esos datos

In [14]:
df_clean=df.dropna()


In [15]:
df_clean.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Ahora que tenemos un dataset limpio, repetimos el proceso

In [16]:
#Escojamos nuestras variables predictoras
predictores = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
#Escojamos que es lo que queremos encontrar
objetivo= 'Outcome'

#Recordemos que en nuestra variable Y va lo que queremos encontrar/predecir.
#Mientras en la vairable X van las variables que usaremos para encontrar a Y
y= df_clean[objetivo].values
X= df_clean[predictores].values

In [17]:
# Crear conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(595, 8)
(149, 8)
(595,)
(149,)


In [18]:
#Se crea el modelo
log_reg = LogisticRegression(solver='lbfgs',max_iter=100)
#se entrena el modelo
log_reg.fit(X_train, y_train)

c:\Users\GlobE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
log_reg.score(X_test, y_test)

0.8187919463087249

Se puede observar que ya obtuvimos un mejor resultado sin nulos usando regresión logistica.

Dicho esto, usaremos el hgbm de nuevo para una mejor comparación

In [20]:
hgbm = HistGradientBoostingClassifier(random_state=42)
hgbm.fit(X_train, y_train)
hgbm.score(X_test, y_test)

0.7919463087248322

Como se puede ver, este modelo tambien presenta una mejora en su score.